<a href="https://colab.research.google.com/github/hrai/M4-Data-Analysis/blob/master/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
import pandas as pd

url='https://raw.githubusercontent.com/hrai/M4-Data-Analysis/master/data/M4_passages_2018-07.csv'
df = pd.read_csv(url)


In [2]:
df.items

df[:3]

,AssetID,FinancialQtrID,Date,IntervalStart,IntervalEnd,Version,VehicleClass,TollPointID,GantryDirection,GantryLocation,GantryGPSLatitude,GantryGPSLongitude,GantryType,TotalVolume
0,M4,201901,2018-07-01,00:00,00:14,3,Car,TS0101,W,Church St Exit,-33.83,151.0,Mainline,310
1,M4,201901,2018-07-01,00:00,00:14,3,Truck,TS0101,W,Church St Exit,-33.83,151.0,Mainline,4
2,M4,201901,2018-07-01,00:00,00:14,3,Car,TS0102,E,Church St Entry,-33.83,151.0,Mainline,120


In [3]:
df.columns

Index(['AssetID', 'FinancialQtrID', 'Date', 'IntervalStart', 'IntervalEnd',
       'Version', 'VehicleClass', 'TollPointID', 'GantryDirection',
       'GantryLocation', 'GantryGPSLatitude', 'GantryGPSLongitude',
       'GantryType', 'TotalVolume'],
      dtype='object')

## Check for null values



In [4]:
df.isnull().values.any()

False

## Rename column

In [7]:
df.rename(columns={'TotalVolume': 'TotalTrafficVolume'}, inplace=True)
df.columns

Index(['AssetID', 'FinancialQtrID', 'Date', 'IntervalStart', 'IntervalEnd',
       'Version', 'VehicleClass', 'TollPointID', 'GantryDirection',
       'GantryLocation', 'GantryGPSLatitude', 'GantryGPSLongitude',
       'GantryType', 'TotalTrafficVolume'],
      dtype='object')

In [43]:
df.values

array([['M4', 201901, '2018-07-01', ..., 151.0, 'Mainline', 310],
       ['M4', 201901, '2018-07-01', ..., 151.0, 'Mainline', 4],
       ['M4', 201901, '2018-07-01', ..., 151.0, 'Mainline', 120],
       ...,
       ['M4', 201901, '2018-07-31', ..., 151.07, 'Ramp', 24],
       ['M4', 201901, '2018-07-31', ..., 151.07, 'Ramp', 5],
       ['M4', 201901, '2018-07-31', ..., 151.07, 'Mainline + G', 17]],
      dtype=object)

## Checking latitute and longitude against GantryLocation

In [63]:
temp_df=df[['GantryLocation','GantryGPSLatitude','GantryGPSLongitude','TollPointID']].groupby(by=['GantryLocation'])
print(temp_df.first())

                            GantryGPSLatitude  GantryGPSLongitude TollPointID
GantryLocation                                                               
Church St Entry                        -33.83              151.00      TS0102
Church St Exit                         -33.83              151.00      TS0101
Hill Rd Entry                          -33.85              151.05      TS1001
Hill Rd Exit                           -33.85              151.05      TS0901
Homebush Bay Dr (NB) Entry             -33.86              151.07      TS1301
Homebush Bay Dr Entry                  -33.86              151.07      TS1401
Homebush Bay Dr Exit                   -33.86              151.07      TS1201
James Ruse Dr Entry                    -33.83              151.02      TS0201
James Ruse Dr Exit                     -33.83              151.02      TS0202
Silverwater Rd Entry                   -33.84              151.04      TS0601
Silverwater Rd Exit                    -33.84              151.0

In [68]:
print(df['AssetID'].unique())
print(df['Version'].unique())
print(df['FinancialQtrID'].unique())
#print(df['Date'].unique())

['M4']
[3]
[201901]


## Drop unwanted columns like latitude and longitude

In [66]:
df.drop(['GantryGPSLatitude', 'GantryGPSLongitude','AssetID', 'Version','TollPointID','FinancialQtrID'], axis = 1) 

,Date,IntervalStart,IntervalEnd,VehicleClass,GantryDirection,GantryLocation,GantryType,TotalVolume
0,2018-07-01,00:00,00:14,Car,W,Church St Exit,Mainline,310
1,2018-07-01,00:00,00:14,Truck,W,Church St Exit,Mainline,4
2,2018-07-01,00:00,00:14,Car,E,Church St Entry,Mainline,120
3,2018-07-01,00:00,00:14,Truck,E,Church St Entry,Mainline,2
4,2018-07-01,00:00,00:14,Car,W,James Ruse Dr Entry,Ramp,23
5,2018-07-01,00:00,00:14,Car,E,James Ruse Dr Exit,Ramp,4
6,2018-07-01,00:00,00:14,Car,E,James Ruse Dr Entry,Ramp,43
7,2018-07-01,00:00,00:14,Car,W,James Ruse Dr Exit,Ramp,81
8,2018-07-01,00:00,00:14,Car,E,Silverwater Rd Exit,Ramp,20
9,2018-07-01,00:00,00:14,Car,W,Silverwater Rd Entry,Ramp,39


In [44]:
df['GantryLocation'].unique()

array(['Church St Exit', 'Church St Entry', 'James Ruse Dr Entry',
       'James Ruse Dr Exit', 'Silverwater Rd Exit',
       'Silverwater Rd Entry', 'Hill Rd Exit', 'Hill Rd Entry',
       'Homebush Bay Dr Exit', 'Homebush Bay Dr (NB) Entry',
       'Homebush Bay Dr Entry'], dtype=object)

In [45]:
df['VehicleClass'].unique()

array(['Car', 'Truck'], dtype=object)

In [14]:
df['GantryType'].unique()

array(['Mainline', 'Ramp', 'Mainline + G'], dtype=object)

In [46]:
group_by_gantry_type = df.groupby(by=['GantryType'])
print(group_by_gantry_type.head(4))


   AssetID  FinancialQtrID  ...    GantryType TotalVolume
0       M4          201901  ...      Mainline         310
1       M4          201901  ...      Mainline           4
2       M4          201901  ...      Mainline         120
3       M4          201901  ...      Mainline           2
4       M4          201901  ...          Ramp          23
5       M4          201901  ...          Ramp           4
6       M4          201901  ...          Ramp          43
7       M4          201901  ...          Ramp          81
22      M4          201901  ...  Mainline + G         266
23      M4          201901  ...  Mainline + G           3
49      M4          201901  ...  Mainline + G         204
50      M4          201901  ...  Mainline + G           1

[12 rows x 14 columns]


In [47]:
car_data_avg = group_by_gantry_type.mean()
car_data_count = group_by_gantry_type.count()

print(car_data_avg)
print(car_data_count)

              FinancialQtrID  Version  ...  GantryGPSLongitude  TotalVolume
GantryType                             ...                                 
Mainline            201901.0      3.0  ...          151.021767   224.135098
Mainline + G        201901.0      3.0  ...          151.070000   181.506783
Ramp                201901.0      3.0  ...          151.042347    50.881179

[3 rows x 5 columns]
              AssetID  FinancialQtrID  ...  GantryGPSLongitude  TotalVolume
GantryType                             ...                                 
Mainline        17269           17269  ...               17269        17269
Mainline + G     5823            5823  ...                5823         5823
Ramp            71856           71856  ...               71856        71856

[3 rows x 13 columns]
